As part of putting this session together, I came up with a few things which might be interesting but which didn't fit the presentation schedule.

# Health Care Research change

Data from the NIH: https://report.nih.gov/categorical_spending.aspx

You can download an "Excel" file from that page, but it's actually the HTML version of an Excel file, so `pandas` can't read it. I opened it in Excel for Mac and saved it as a CSV.

Note that Mac Excel files use the `MacRoman` encoding. 

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv('RCDCFundingSummary030516.csv',encoding='MacRoman', skiprows=1, header=None, 
                 names=['area', '2012', '2013', '2014', '2015', '2016', '2017'])
df.head()

,area,2012,2013,2014,2015,2016,2017
0,Acquired Cognitive Impairment,+,+,+,$798,$832,$832
1,Acute Respiratory Distress Syndrome,$98,$95,$85,$108,$112,$112
2,Adolescent Sexual Activity,$76,$70,$68,$85,$88,$88
3,Agent Orange & Dioxin,$8,$10,$8,$9,$9,$9
4,Aging,"$2,593","$2,429","$2,556","$2,698","$2,807","$2,807"


## Strip formatting characters

Eyeballing the data (and, to be honest, experience trying to load it) shows that there are a number of values which will get in our way. There are columns with '+' and '-' characters, and all of the numeric columns are formatted as currency, with '$' and ',' characters which will get in our way. Let's get rid of them.

In [10]:
df = df.replace('[\$,+\-]','',regex=True) # strip $|,|+|- from all cells. 
df.head()

,area,2012,2013,2014,2015,2016,2017
0,Acquired Cognitive Impairment,,,,798,832,832
1,Acute Respiratory Distress Syndrome,98,95,85,108,112,112
2,Adolescent Sexual Activity,76,70,68,85,88,88
3,Agent Orange & Dioxin,8,10,8,9,9,9
4,Aging,2593,2429,2556,2698,2807,2807


## Deal with blanks

When you want to convert a column of values as numbers, the blanks cause an error. Let's use `replace` again to put a number-safe placeholder for "not a number" in there, and then convert all of the number columns to `float`. (You may look at them and think of using `int` since there are no decimals, but `np.nan` doesn't work with integers.) 

In [15]:
df = df.replace('^\s*$',np.nan,regex=True)
for col in df.columns[1:]: # skip the first column
    df[col] = df[col].astype('float')

df.describe()    

,2012,2013,2014,2015,2016,2017
count,241.000000,241.000000,248.000000,265.000000,264.000000,264.000000
mean,612.676349,592.344398,601.201613,588.094340,618.159091,621.026515
std,1334.687692,1303.804968,1335.118554,1326.098431,1386.499550,1397.501190
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,24.000000,25.000000,24.000000,25.000000,27.500000,27.500000
50%,115.000000,114.000000,114.500000,108.000000,113.500000,113.500000
75%,477.000000,437.000000,426.500000,424.000000,436.500000,436.500000
max,10951.000000,10604.000000,11087.000000,11366.000000,11867.000000,11867.000000


You'll see that the value for `count` is different for each column. That's because `NaN` values are excluded from the cound. We can see that things are inconsistent, so we may want to just drop all of the `NaN` values for now.

There are also some cases where the spending in 2012 is zero, so computing a percent change over time is complicated, so we'll just skip those too.

In [18]:
df = df.dropna()
df = df[df['2012'] > 0]


237

In [21]:
df['change'] = ( (df['2017'] - df['2012'])/df['2012'])


In [22]:
df.sort_values(by='change').head()

,area,2012,2013,2014,2015,2016,2017,change
68,Diethylstilbestrol (DES),3,1,1,1,1,1,-0.666667
152,Mind and Body,533,210,211,217,226,226,-0.575985
90,Foodborne Illness 7/,257,229,232,105,109,109,-0.575875
215,Small Pox,40,30,24,17,18,18,-0.550000
41,Cerebral Palsy,42,18,21,20,21,21,-0.500000


In [24]:
df[df['area'] == 'Fibromyalgia']

,area,2012,2013,2014,2015,2016,2017,change
88,Fibromyalgia,13,11,10,8,9,9,-0.307692


In [25]:
df.sum()

area      Acute Respiratory Distress SyndromeAdolescent ...
2012                                                 147655
2013                                                 142028
2014                                                 147437
2015                                                 151019
2016                                                 157799
2017                                                 158556
change                                              12.5506
dtype: object

In [32]:
df[df['2015'] >= 7][df['2015'] <= 10].sort_values(by='2015').tail(10)

/Users/germuska/.virtualenvs/pandas-nicar-2016/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,area,2012,2013,2014,2015,2016,2017,change
70,Digestive Diseases (Gallbladder),11,10,9,8,8,8,-0.272727
85,Facioscapulohumeral Muscular Dystrophy,5,5,7,8,9,9,0.800000
88,Fibromyalgia,13,11,10,8,9,9,-0.307692
3,Agent Orange & Dioxin,8,10,8,9,9,9,0.125000
158,Myotonic Dystrophy,10,10,9,9,10,10,0.000000
12,Anorexia,14,11,11,10,10,10,-0.285714
81,Endometriosis,9,7,7,10,11,11,0.222222
87,Fibroid Tumors (Uterine),14,10,9,10,10,10,-0.285714
134,Interstitial Cystitis,10,10,9,10,11,11,0.100000
217,Spina Bifida,12,10,10,10,10,10,-0.166667
